In [ ]:
#Packages
import os
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [ ]:
#Get Data
csv_dir = '../../SEC Trackman Data/'
all_files = [os.path.join(csv_dir, f) for f in os.listdir(csv_dir) if f.endswith('.csv')]

df_list = []

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

all_pitches = pd.concat(df_list, ignore_index=True)

print(all_pitches.shape)
print(all_pitches.columns)

In [ ]:
#Get all sliders
bbs = all_pitches[all_pitches['TaggedPitchType'].isin(['Slider'])]
print(bbs.shape)

In [ ]:
#Whiffs
bbs['whiff'] = np.where(bbs['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

In [ ]:
bbs_X = bbs[preds]
bbs_y = bbs['whiff']

In [ ]:
#Train Model
X_train, X_test, y_train, y_test = train_test_split(bbs_X, bbs_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_bbs = bbs[bbs['PitcherTeam']=='AUB_TIG']

In [ ]:
# Get unique values
unique_values = bbs['Pitcher'].unique()
aub_unique_values = aub_bbs['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: bbs[bbs['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: bbs[bbs['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_bbs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bbs_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} Slider stuff+: {np.mean(stuff * 100)}")

In [ ]:
#Stuff+ for selected Auburn pitchers Slider
for pitcher in aub_unique_values:
    aub_bbs_stuff(pitcher)

Now, let's do splits

In [ ]:
#Subset sliders into platoon splits
bbs_rr = bbs[(bbs['PitcherThrows'] == 'Right') & (bbs['BatterSide'] == 'Right')]
bbs_rl = bbs[(bbs['PitcherThrows'] == 'Right') & (bbs['BatterSide'] == 'Left')]
bbs_lr = bbs[(bbs['PitcherThrows'] == 'Left') & (bbs['BatterSide'] == 'Right')]
bbs_ll = bbs[(bbs['PitcherThrows'] == 'Left') & (bbs['BatterSide'] == 'Left')]

print(bbs_rr.shape)
print(bbs_rl.shape)
print(bbs_lr.shape)
print(bbs_ll.shape)

In [ ]:
#Whiffs
bbs_rr['whiff'] = np.where(bbs_rr['PitchCall']=='StrikeSwinging',1,0)
bbs_rl['whiff'] = np.where(bbs_rl['PitchCall']=='StrikeSwinging',1,0)
bbs_lr['whiff'] = np.where(bbs_lr['PitchCall']=='StrikeSwinging',1,0)
bbs_ll['whiff'] = np.where(bbs_ll['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
#Predictor Variables
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

Righty vs. Righty Split

In [ ]:
bbs_rr_X = bbs_rr[preds]
bbs_rr_y = bbs_rr['whiff']

In [ ]:
#Train Model: RHP vs RHB
X_train, X_test, y_train, y_test = train_test_split(bbs_rr_X, bbs_rr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
y_pred_prob = model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, predictions, multi_class='ovr')

print(f'ROC AUC: {roc_auc:.4f}')

# Plotting ROC Curve for one class (e.g., class 0)
fpr, tpr, _ = roc_curve(y_test == 1, y_pred_prob)
plt.plot(fpr, tpr, label='Class 0 ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
#Auburn Subset
aub_bbs_rr = bbs_rr[bbs_rr['PitcherTeam']=='AUB_TIG']
print(aub_bbs_rr.shape)

In [ ]:
# Get unique values
unique_values = bbs_rr['Pitcher'].unique()
aub_unique_values = aub_bbs_rr['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)


# Create a dictionary to store subsets
subsets = {value: bbs_rr[bbs_rr['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: bbs_rr[bbs_rr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rr_bbs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bbs_rr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} slider rr stuff+: {np.mean(stuff * 100)}")

    


In [ ]:
#Stuff+ for selected Auburn pitchers Slider in RHP v RHB split
for pitcher in aub_unique_values:
    aub_rr_bbs_stuff(pitcher)

Righty vs. Lefty Split

In [ ]:
bbs_rl_X = bbs_rl[preds]
bbs_rl_y = bbs_rl['whiff']

In [ ]:
#Train Model: RHP vs LHB
X_train, X_test, y_train, y_test = train_test_split(bbs_rl_X, bbs_rl_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_bbs_rl = bbs_rl[bbs_rl['PitcherTeam']=='AUB_TIG']
print(aub_bbs_rl.shape)

In [ ]:
# Get unique values
unique_values = bbs_rl['Pitcher'].unique()
aub_unique_values = aub_bbs_rl['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: bbs_rl[bbs_rl['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: bbs_rl[bbs_rl['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rl_bbs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bbs_rl_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} slider rl stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers Slider in RHP v LHB split
for pitcher in aub_unique_values:
    aub_rl_bbs_stuff(pitcher)

Lefty vs. Righty Split

In [ ]:
bbs_lr_X = bbs_lr[preds]
bbs_lr_y = bbs_lr['whiff']

In [ ]:
#Train Model: LHP vs RHB
X_train, X_test, y_train, y_test = train_test_split(bbs_lr_X, bbs_lr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_bbs_lr = bbs_lr[bbs_lr['PitcherTeam']=='AUB_TIG']
print(aub_bbs_lr.shape)

In [ ]:
# Get unique values
unique_values = bbs_lr['Pitcher'].unique()
aub_unique_values = aub_bbs_lr['Pitcher'].unique()


#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: bbs_lr[bbs_lr['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: bbs_lr[bbs_lr['Pitcher'] == value] for value in aub_unique_values}


In [ ]:
def aub_lr_bbs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bbs_lr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} slider lr stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers Slider in LHP v RHB split
for pitcher in aub_unique_values:
    aub_lr_bbs_stuff(pitcher)

Lefty vs. Lefty Split

In [ ]:
bbs_ll_X = bbs_ll[preds]
bbs_ll_y = bbs_ll['whiff']

In [ ]:
#Train Model: LHP vs LHB
X_train, X_test, y_train, y_test = train_test_split(bbs_ll_X, bbs_ll_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_bbs_ll = bbs_ll[bbs_ll['PitcherTeam']=='AUB_TIG']
print(aub_bbs_ll.shape)

In [ ]:
# Get unique values
unique_values = bbs_ll['Pitcher'].unique()
aub_unique_values = aub_bbs_ll['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: bbs_ll[bbs_ll['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: bbs_ll[bbs_ll['Pitcher'] == value] for value in aub_unique_values}



In [ ]:
def aub_ll_bbs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bbs_ll_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} slider ll stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers Slider in LHP v LHB split
for pitcher in aub_unique_values:
    aub_ll_bbs_stuff(pitcher)